In [ ]:
## Install required packages
#%pip install qiskit-ibm-runtime

In [ ]:
from qiskit_ibm_runtime.fake_provider import FakeProviderForBackendV2
from qiskit_aer.noise import NoiseModel
from IPython.display import display, Markdown, Latex
import csv
import os
from itertools import combinations

In [ ]:
%mkdir -p hardware

In [ ]:
def extract_hardware_data():
    """Extract hardware data from all backends and qubits."""
    
    all_data = []

    for backend in FakeProviderForBackendV2().backends():
        # Extract noise model from real hardware (well, fake in this case!)
        noise_model = NoiseModel.from_backend(backend)

        # Get detailed error rates for ALL qubits (no limitation)
        properties = backend.properties()
        #print(f"{backend.name.replace("fake_", "").title()} - {noise_model.noise_instructions}")
        n_qubits = backend.configuration().n_qubits

        for qubit in range(n_qubits):
            readout_error = properties.readout_error(qubit)

            # Store data for table
            all_data.append({
                'Backend': backend.name.replace("fake_", "").title(),
                'Num_Qubits': n_qubits,
                'Qubit': qubit,
                'Readout_Error_Percent': round(readout_error * 100, 2),
                'Basis_Gates_Count': len(noise_model.basis_gates) if noise_model.basis_gates else 0,
                'Noise_Instructions_Count': len(noise_model.noise_instructions)
            })

    return all_data


In [ ]:
def data_to_latex_table(data):
    """Convert data to LaTeX table format."""
    
    latex_table = """\\begin{table}[htbp]
\\centering
\\caption{Quantum Hardware Characteristics}
\\label{tab:hardware_data}
\\begin{tabular}{|l|c|c|c|c|}
\\hline
\\textbf{Backend} & \\textbf{Qubit} & \\textbf{Readout Error} & \\textbf{Basis Gates Count} & \\textbf{Noise Instructions Count} \\\\
\\hline
"""

    for row in data:
        latex_table += f"{row['Backend']} & {row['Qubit']} & {row['Readout_Error_Percent']}\\% & {row['Basis_Gates_Count']} & {row['Noise_Instructions_Count']} \\\\ \\hline \n"

    latex_table += """
\\end{tabular}
\\end{table}"""

    return latex_table

def save_data_to_csv(data, filepath='hardware/hardware.csv'):
    """Save data to CSV file."""
    
    # Ensure hardware directory exists
    os.makedirs('hardware', exist_ok=True)
    
    # Save to CSV
    with open(filepath, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Backend', 'Qubit', 'Readout_Error_Percent', 'Basis_Gates_Count', 'Noise_Instructions_Count']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, extrasaction="ignore")
        
        # Write header
        writer.writeheader()
        
        # Write data rows
        for row in data:
            writer.writerow(row)

def run_hardware_analysis(data):
    """Extract data, print LaTeX table and save CSV."""
    
    # Convert to LaTeX and print
    latex_output = data_to_latex_table(data)
    print(latex_output)
    
    # Save to CSV
    save_data_to_csv(data)
    print(f"\nData saved to: hardware/hardware.csv")
    print(f"Total rows: {len(data)}")



In [ ]:
def find_optimal_qubit_groups(data, max_group_size=5, min_group_size=2):
    """Find optimal qubit groups with lowest readout errors per backend."""
    
    # Group data by backend
    backend_data = {}
    for row in data:
        backend = row['Backend']
        if backend not in backend_data:
            backend_data[backend] = []
        backend_data[backend].append({
            'qubit': row['Qubit'],
            'error': row['Readout_Error_Percent']
        })
    
    optimal_groups = {}
    
    for backend, qubits in backend_data.items():
        # Sort qubits by readout error (lowest first)
        qubits_sorted = sorted(qubits, key=lambda x: x['error'])
        
        best_groups = []
        
        # Try different group sizes (larger groups preferred)
        for group_size in range(max_group_size, min_group_size - 1, -1):
            if len(qubits_sorted) < group_size:
                continue
                
            # Find all possible combinations of this size
            for combo in combinations(qubits_sorted[:min(10, len(qubits_sorted))], group_size):
                # Calculate statistics for this group
                errors = [q['error'] for q in combo]
                avg_error = sum(errors) / len(errors)
                min_error = min(errors)
                max_error = max(errors)
                
                group_info = {
                    'qubits': [q['qubit'] for q in combo],
                    'errors': errors,
                    'avg_error': round(avg_error, 3),
                    'min_error': round(min_error, 3),
                    'max_error': round(max_error, 3),
                    'size': group_size
                }
                
                best_groups.append(group_info)
        
        # Sort by group size (descending) then by maximum error (ascending)
        best_groups.sort(key=lambda x: (-x['size'], x['max_error']))
        
        # Take the best group
        if best_groups:
            optimal_groups[backend] = best_groups[0]
    
    return optimal_groups

def optimal_groups_to_latex(optimal_groups):
    """Convert optimal qubit groups to LaTeX table."""
    
    latex_table = """\\begin{table}[htbp]
\\centering
\\caption{Optimal Qubit Groups by Backend (Lowest Readout Errors)}
\\label{tab:optimal_qubit_groups}
\\begin{tabular}{|l|c|c|c|c|c|c|}
\\hline
\\textbf{Backend} & \\textbf{Group Size} & \\textbf{Qubits} & \\textbf{Individual Errors} & \\textbf{Min Error} & \\textbf{Max Error} & \\textbf{Avg Error} \\\\
\\hline
"""

    # Sort backends by maximum error for better presentation
    sorted_backends = sorted(optimal_groups.items(), key=lambda x: x[1]['max_error'])
    
    for backend, group_info in sorted_backends:
        qubits_str = ', '.join(map(str, group_info['qubits']))
        errors_str = ', '.join(f"{e}\\%" for e in group_info['errors'])
        
        latex_table += f"{backend} & {group_info['size']} & {qubits_str} & {errors_str} & {group_info['min_error']}\\% & {group_info['max_error']}\\% & {group_info['avg_error']}\\% \\\\ \\hline \n"

    latex_table += """
\\end{tabular}
\\end{table}"""

    return latex_table

def save_optimal_groups_to_csv(optimal_groups, filepath='hardware/optimal_qubit_groups.csv'):
    """Save optimal qubit groups to CSV file."""
    
    # Ensure hardware directory exists
    os.makedirs('hardware', exist_ok=True)
    
    # Prepare data for CSV
    csv_data = []
    for backend, group_info in optimal_groups.items():
        csv_data.append({
            'Backend': backend,
            'Group Size': group_info['size'],
            'Qubits': ';'.join(map(str, group_info['qubits'])),  # semicolon separated for CSV
            'Individual Errors': ';'.join(map(str, group_info['errors'])),
            'Min Error': group_info['min_error'],
            'Max Error': group_info['max_error'],
            'Average Error': group_info['avg_error']
        })
    
    # Save to CSV
    with open(filepath, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Backend', 'Group Size', 'Qubits', 'Individual Errors', 'Min Error', 'Max Error', 'Average Error']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, extrasaction="ignore")
        
        # Write header
        writer.writeheader()
        
        # Write data rows
        for row in csv_data:
            writer.writerow(row)

def run_optimal_qubit_analysis(data, max_group_size=5, min_group_size=2):
    """Find and display optimal qubit groups."""
       
    # Find optimal groups
    optimal_groups = find_optimal_qubit_groups(data, max_group_size, min_group_size)
    
    # Convert to LaTeX and print
    latex_output = optimal_groups_to_latex(optimal_groups)
    print(latex_output)
    
    # Save to CSV
    save_optimal_groups_to_csv(optimal_groups)
    print(f"\nOptimal groups saved to: hardware/optimal_qubit_groups.csv")
    print(f"Found optimal groups for {len(optimal_groups)} backends")
    
    # Print summary
    print("\nSummary of optimal groups:")
    for backend, group_info in sorted(optimal_groups.items(), key=lambda x: x[1]['max_error']):
        qubits_str = ', '.join(map(str, group_info['qubits']))
        print(f"{backend}: {group_info['size']} qubits [{qubits_str}] - Min: {group_info['min_error']}%, Max: {group_info['max_error']}%, Avg: {group_info['avg_error']}%")


In [ ]:
def print_hardware_summary_latex(data):
    """Print a summary table showing backend overview in LaTeX format."""
    
    if not data:
        print("No data to display")
        return
    
    # Group data by backend to create summary
    backend_summary = {}
    for row in data:
        backend = row['Backend']
        if backend not in backend_summary:
            backend_summary[backend] = {
                'num_qubits': row['Num_Qubits'],
                'avg_readout_error': [],
                'basis_gates': row['Basis_Gates_Count'],
                'noise_instructions': row['Noise_Instructions_Count']
            }
        backend_summary[backend]['avg_readout_error'].append(row['Readout_Error_Percent'])
    
    # Calculate averages
    for backend in backend_summary:
        errors = backend_summary[backend]['avg_readout_error']
        backend_summary[backend]['avg_readout_error'] = round(sum(errors) / len(errors), 2)
    
    # LaTeX summary table
    latex_output = r"""
\begin{table}[htbp]
\centering
\caption{Quantum Backend Summary}
\label{tab:backend_summary}
\begin{tabular}{|l|c|c|c|c|}
\hline
\textbf{Backend} & \textbf{Qubits} & \textbf{Avg. Readout Error (\%)} & \textbf{Basis Gates} & \textbf{Noise Instructions} \\
\hline
"""
    
    # Add summary rows
    for backend, info in backend_summary.items():
        latex_output += f"{backend} & {info['num_qubits']} & {info['avg_readout_error']} & {info['basis_gates']} & {info['noise_instructions']} \\\\ \\hline \n"
    
    # LaTeX table footer
    latex_output += r"""
\end{tabular}
\end{table}
"""
    
    print(latex_output)


In [ ]:
def get_cx_error_from_backend(backend, qubit1, qubit2):
    """
    Get CX gate error rate between two specific qubits using main methods.
    
    Args:
        backend: Qiskit backend
        qubit1, qubit2: Qubit indices
        
    Returns:
        dict: CX gate information or None if not available
    """
    try:
        properties = backend.properties()
        
        # Method 1: Direct gate_error method (most reliable)
        try:
            error_rate = properties.gate_error('cx', [qubit1, qubit2])
            gate_time = properties.gate_length('cx', [qubit1, qubit2])
            
            # Skip if error rate is zero (indicates unavailable CNOT)
            if error_rate == 0:
                return None
            
            return {
                'error_rate': error_rate,
                'error_percent': round(error_rate * 100, 2),  # Changed to 2 digits
                'gate_time': gate_time,
                'method': 'properties_direct'
            }
        except:
            pass
        
        # Method 2: Search through all gates in properties
        for gate in properties.gates:
            if gate.gate == 'cx' and set(gate.qubits) == {qubit1, qubit2}:
                error_rate = gate.parameters[0].value  # Gate error is usually first parameter
                
                # Skip if error rate is zero (indicates unavailable CNOT)
                if error_rate == 0:
                    return None
                
                gate_time = None
                
                # Try to find gate time
                for param in gate.parameters:
                    if param.name == 'gate_time':
                        gate_time = param.value
                        break
                
                return {
                    'error_rate': error_rate,
                    'error_percent': round(error_rate * 100, 2),  # Changed to 2 digits
                    'gate_time': gate_time,
                    'method': 'properties_search'
                }
        
        # Skip Method 3 - we ignore pairs without connectivity
            
    except Exception as e:
        pass
    
    return None


def analyze_group_cx_gates(backend, qubit_group):
    """
    Analyze all possible CX gate pairs within a qubit group.
    Only includes pairs with actual connectivity and non-zero error rates.
    
    Args:
        backend: Qiskit backend object
        qubit_group: List of qubit indices
        
    Returns:
        dict: CX gate analysis for the group
    """
    cx_analysis = {
        'total_pairs': 0,
        'available_cx_pairs': 0,
        'cx_gates': {},
        'statistics': {}
    }
    
    # Get all possible pairs within the group
    all_pairs = list(combinations(qubit_group, 2))
    cx_analysis['total_pairs'] = len(all_pairs)
    
    cx_errors = []
    cx_times = []
    
    for q1, q2 in all_pairs:
        # Try both directions for CX gate
        cx_info = get_cx_error_from_backend(backend, q1, q2)
        if not cx_info:
            cx_info = get_cx_error_from_backend(backend, q2, q1)
        
        # Only count pairs with valid connectivity and non-zero error rates
        if cx_info and cx_info['error_rate'] is not None and cx_info['error_rate'] > 0:
            cx_analysis['available_cx_pairs'] += 1
            cx_analysis['cx_gates'][(q1, q2)] = cx_info
            cx_errors.append(cx_info['error_rate'])
            if cx_info['gate_time'] is not None:
                cx_times.append(cx_info['gate_time'])
    
    # Calculate statistics
    if cx_errors:
        cx_analysis['statistics'] = {
            'avg_cx_error': round(sum(cx_errors) / len(cx_errors), 6),
            'min_cx_error': round(min(cx_errors), 6),
            'max_cx_error': round(max(cx_errors), 6),
            'avg_cx_error_percent': round(sum(cx_errors) / len(cx_errors) * 100, 2),  # Changed to 2 digits
            'connectivity_ratio': round(cx_analysis['available_cx_pairs'] / cx_analysis['total_pairs'], 3)
        }
        
        if cx_times:
            cx_analysis['statistics']['avg_gate_time'] = round(sum(cx_times) / len(cx_times), 9)
            cx_analysis['statistics']['min_gate_time'] = round(min(cx_times), 9)
            cx_analysis['statistics']['max_gate_time'] = round(max(cx_times), 9)
    
    return cx_analysis


def calculate_total_error(readout_errors, cx_error_rate):
    """
    Calculate total error as sum of two qubit readout errors plus CX error.
    
    Args:
        readout_errors: List of readout error percentages for qubits in the pair
        cx_error_rate: CX gate error rate (as percentage)
        
    Returns:
        float: Total error percentage
    """
    return sum(readout_errors) + cx_error_rate


def find_optimal_qubit_groups_with_cx(data, max_group_size=5, min_group_size=2):
    """
    Find optimal qubit groups with lowest total errors (readout + CX) and analyze CX gates within groups.
    
    Args:
        data: Hardware data from extract_hardware_data()
        max_group_size: Maximum size of qubit groups
        min_group_size: Minimum size of qubit groups
        
    Returns:
        dict: Optimal groups with CX gate analysis
    """
    
    # Group data by backend
    backend_data = {}
    backends_map = {}  # Store backend objects
    
    # Get backend objects
    provider = FakeProviderForBackendV2()
    for backend_obj in provider.backends():
        backend_name = backend_obj.name.replace("fake_", "").title()
        backends_map[backend_name] = backend_obj
    
    for row in data:
        backend = row['Backend']
        if backend not in backend_data:
            backend_data[backend] = []
        backend_data[backend].append({
            'qubit': row['Qubit'],
            'error': row['Readout_Error_Percent']
        })
    
    optimal_groups = {}
    
    for backend, qubits in backend_data.items():
        if backend not in backends_map:
            print(f"Warning: Backend {backend} not found in provider")
            continue
            
        backend_obj = backends_map[backend]
        
        # Sort qubits by readout error (lowest first)
        qubits_sorted = sorted(qubits, key=lambda x: x['error'])
        
        best_groups = []
        
        # Try different group sizes (larger groups preferred)
        for group_size in range(max_group_size, min_group_size - 1, -1):
            if len(qubits_sorted) < group_size:
                continue
                
            # Find all possible combinations of this size
            for combo in combinations(qubits_sorted[:min(10, len(qubits_sorted))], group_size):
                # Calculate readout error statistics for this group
                errors = [q['error'] for q in combo]
                avg_error = sum(errors) / len(errors)
                min_error = min(errors)
                max_error = max(errors)
                total_readout_error = sum(errors)  # Add total readout error calculation
                
                qubit_indices = [q['qubit'] for q in combo]
                
                # Analyze CX gates within this group
                cx_analysis = analyze_group_cx_gates(backend_obj, qubit_indices)
                
                # Calculate total error (readout + CX) for ranking
                total_error = 0
                if cx_analysis['statistics'] and cx_analysis['cx_gates']:
                    # For pairs, calculate total error as sum of readout errors + CX error
                    if group_size == 2:
                        cx_error_percent = cx_analysis['statistics']['avg_cx_error_percent']
                        total_error = sum(errors) + cx_error_percent
                    else:
                        # For larger groups, use average approach
                        total_error = avg_error * group_size + cx_analysis['statistics']['avg_cx_error_percent']
                else:
                    # If no CX connectivity, use a high penalty
                    total_error = float('inf')
                
                group_info = {
                    'qubits': qubit_indices,
                    'readout_errors': [round(e, 2) for e in errors],  # Changed to 2 digits
                    'avg_readout_error': round(avg_error, 2),  # Changed to 2 digits
                    'min_readout_error': round(min_error, 2),  # Changed to 2 digits
                    'max_readout_error': round(max_error, 2),  # Changed to 2 digits
                    'total_readout_error': round(total_readout_error, 2),  # Add total readout error
                    'size': group_size,
                    'cx_analysis': cx_analysis,
                    'total_error': round(total_error, 2) if total_error != float('inf') else None
                }
                
                best_groups.append(group_info)
        
        # Sort by minimum total error (ascending), then connectivity ratio (descending), then group size (descending)
        best_groups = [g for g in best_groups if g['total_error'] is not None]  # Remove groups without connectivity
        best_groups.sort(key=lambda x: (
            x['total_error'],
            -x['cx_analysis']['statistics'].get('connectivity_ratio', 0),
            -x['size']
        ))
        
        # Take the best group
        if best_groups:
            optimal_groups[backend] = best_groups[0]
    
    return optimal_groups


def get_backend_info(backend_obj):
    """
    Get backend information including number of qubits and availability date if possible.
    
    Args:
        backend_obj: Qiskit backend object
        
    Returns:
        dict: Backend information
    """
    info = {
        'name': backend_obj.name,
        'num_qubits': backend_obj.configuration().n_qubits,
        'availability_date': None,
        'description': None
    }
    
    try:
        # Try to get description from backend
        config = backend_obj.configuration()
        if hasattr(config, 'description'):
            info['description'] = config.description
        
        # Try to get backend version or date information
        if hasattr(config, 'backend_version'):
            info['backend_version'] = config.backend_version
            
        # Some backends might have date information in their properties
        if hasattr(backend_obj, 'properties') and backend_obj.properties():
            props = backend_obj.properties()
            if hasattr(props, 'last_update_date'):
                info['last_update_date'] = props.last_update_date
                
    except Exception as e:
        pass
    
    return info

def print_backend_summary(backends_map):
    """Print summary of all backends with their specifications."""
    
    print("\n" + "="*80)
    print("BACKEND SPECIFICATIONS SUMMARY")
    print("="*80)
    
    backend_info_list = []
    
    for backend_name, backend_obj in backends_map.items():
        info = get_backend_info(backend_obj)
        backend_info_list.append(info)
    
    # Sort by number of qubits (ascending)
    backend_info_list.sort(key=lambda x: x['num_qubits'])
    
    print(f"{'Backend Name':<20} {'Qubits':<8} {'Description'}")
    print("-" * 80)
    
    for info in backend_info_list:
        backend_display = info['name'].replace('fake_', '').title()
        description = info.get('description', 'N/A')
        if description and len(description) > 40:
            description = description[:37] + "..."
        
        print(f"{backend_display:<20} {info['num_qubits']:<8} {description}")
    
    print(f"\nTotal backends analyzed: {len(backend_info_list)}")


def save_to_csv(optimal_groups, backends_map, filename='hardware/optimal_qubit_groups.csv'):
    """Save optimal groups analysis to CSV file with backend info, sorted by Total_Readout_Error_Percent."""
    
    # Prepare data for sorting
    rows_data = []
    
    for backend, group_info in optimal_groups.items():
        cx_stats = group_info['cx_analysis']['statistics']
        cx_gates = group_info['cx_analysis']['cx_gates']
        
        # Get backend info
        backend_obj = backends_map.get(backend)
        backend_info = get_backend_info(backend_obj) if backend_obj else {}
        
        # Format CX gate pairs for CSV
        cx_pairs_str = '; '.join([f"{pair[0]}-{pair[1]}({info['error_percent']}%)" 
                                for pair, info in cx_gates.items()])
        
        row = {
            'Backend': backend,
            'Backend_Qubits': backend_info.get('num_qubits', 'N/A'),
            'Group_Size': group_info['size'],
            'Qubits': ','.join(map(str, group_info['qubits'])),
            'Total_Readout_Error_Percent': group_info['total_readout_error'],
            'Avg_Readout_Error_Percent': group_info['avg_readout_error'],
            'Min_Readout_Error_Percent': group_info['min_readout_error'],
            'Max_Readout_Error_Percent': group_info['max_readout_error'],
            'Total_Pairs': group_info['cx_analysis']['total_pairs'],
            'Available_CX_Pairs': group_info['cx_analysis']['available_cx_pairs'],
            'Connectivity_Ratio': cx_stats.get('connectivity_ratio', 0),
            'Avg_CX_Error_Percent': cx_stats.get('avg_cx_error_percent', 0),
            'Min_CX_Error_Percent': round(cx_stats.get('min_cx_error', 0) * 100, 2) if cx_stats.get('min_cx_error') else 0,
            'Max_CX_Error_Percent': round(cx_stats.get('max_cx_error', 0) * 100, 2) if cx_stats.get('max_cx_error') else 0,
            'Total_Error_Percent': group_info['total_error'],
            'Available_CX_Gate_Pairs': cx_pairs_str,
            'Backend_Description': backend_info.get('description', 'N/A')
        }
        
        rows_data.append(row)
    
    # Sort by Total_Readout_Error_Percent (ascending order - lowest error first)
    rows_data.sort(key=lambda x: float(x['Total_Readout_Error_Percent']) if x['Total_Readout_Error_Percent'] != 'N/A' else float('inf'))
    
    # Write to CSV
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = [
            'Backend', 'Backend_Qubits', 'Group_Size', 'Qubits', 'Total_Readout_Error_Percent',
            'Avg_Readout_Error_Percent', 'Min_Readout_Error_Percent', 'Max_Readout_Error_Percent',
            'Total_Pairs', 'Available_CX_Pairs', 'Connectivity_Ratio',
            'Avg_CX_Error_Percent', 'Min_CX_Error_Percent', 'Max_CX_Error_Percent',
            'Total_Error_Percent', 'Available_CX_Gate_Pairs', 'Backend_Description'
        ]
        
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        # Write sorted data
        for row in rows_data:
            writer.writerow(row)
    
    print(f"\n✅ Results saved to {filename}")

def print_cx_comparison_latex(optimal_groups, backends_map):
    """Print LaTeX table comparing CX gate performance across backends."""
    
    latex_output = r"""
\begin{table}[htbp]
\centering
\caption{Optimal Qubit Groups: CX Gate Performance Comparison (Ordered by Total Error)}
\label{tab:cx_comparison}
\begin{tabular}{|l|c|c|c|c|c|c|c|c|}
\hline
\textbf{Backend} & \textbf{Total Qubits} & \textbf{Group Size} & \textbf{Connectivity} & \textbf{Total Readout (\%)} & \textbf{Avg CX Error (\%)} & \textbf{Total Error (\%)} & \textbf{Qubits} \\
\hline
"""
    
    # Sort by total error for LaTeX table
    sorted_groups = sorted(optimal_groups.items(), key=lambda x: x[1]['total_error'] or float('inf'))
    
    for backend, group_info in sorted_groups:
        # Get backend info
        backend_obj = backends_map.get(backend)
        backend_info = get_backend_info(backend_obj) if backend_obj else {}
        
        cx_stats = group_info['cx_analysis']['statistics']
        connectivity = cx_stats.get('connectivity_ratio', 0) * 100
        avg_cx_error = cx_stats.get('avg_cx_error_percent', 0)
        total_readout = group_info['total_readout_error']  # Use total readout instead of avg
        total_error = group_info['total_error']
        qubits_str = ','.join(map(str, group_info['qubits']))
        total_qubits = backend_info.get('num_qubits', 'N/A')
        
        latex_output += f"{backend} & {total_qubits} & {group_info['size']} & {connectivity:.1f}\\% & {total_readout:.2f} & {avg_cx_error:.2f} & {total_error:.2f} & {qubits_str} \\\\ \\hline \n"
    
    latex_output += r"""\end{tabular}
\end{table}
"""
    
    print("\n" + "="*50)
    print("LATEX TABLE - CX COMPARISON (Ordered by Total Error)")
    print("="*50)
    print(latex_output)


def run_optimal_qubit_analysis_cx(data):
    """Run the complete analysis with all improvements."""
    
    # Get backend objects for info extraction
    provider = FakeProviderForBackendV2()
    backends_map = {}
    for backend_obj in provider.backends():
        backend_name = backend_obj.name.replace("fake_", "").title()
        backends_map[backend_name] = backend_obj

    # Print backend specifications summary
    print_backend_summary(backends_map)
    
    # Find optimal groups with CX analysis
    optimal_groups = find_optimal_qubit_groups_with_cx(data, max_group_size=4, min_group_size=2)
    
    # Print LaTeX comparison table
    print_cx_comparison_latex(optimal_groups, backends_map)
    
    # Save to CSV
    save_to_csv(optimal_groups, backends_map)
    
    return optimal_groups

In [ ]:
data = extract_hardware_data()

In [ ]:
run_hardware_analysis(data)
run_optimal_qubit_analysis(data, max_group_size=4, min_group_size=2)
print_hardware_summary_latex(data)
run_optimal_qubit_analysis_cx(data)